In [ ]:
from aimu.models import OllamaClient

model_client = OllamaClient(OllamaClient.MODEL_LLAMA_3_1_8B)
model_client.model_id

In [ ]:
model_client.generate("What is the capital of France?")

In [ ]:
# For generate_kwargs, see https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values

model_client.generate(
    "How do you say 'it's a beautuiful day' in French?",
    generate_kwargs={
        "temperature": 1.0,
        "top_p": 0.9,
        "top_k": 40,
        "max_tokens": 64,
        "repeat_penalty": 1.1,
    },
)

In [ ]:
response = model_client.generate_streamed(
    "What are the lyrics of La Marseillaise in English? First verse. No commentary.",
    generate_kwargs={
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 40,
        "max_tokens": 1024,
        "repeat_penalty": 1.1,
    },
)

for chunk in response:
    print(chunk, end="", flush=True)

In [ ]:
model_client.chat(
    {
        "role": model_client.system_role,
        "content": "You are a helpful assistant that answers questions using pirate speak.",
    }
)

In [ ]:
model_client.chat({"role": "user", "content": "What is the capital of France?"})

In [ ]:
model_client.messages